## Deliverable 2. Create a Customer Travel Destinations Map.

In [1]:
# Dependencies and Setup
import pandas as pd
import requests
import gmaps

# Import API key
from config import g_key

# Configure gmaps API key
gmaps.configure(api_key=g_key)

In [2]:
# 1. Import the WeatherPy_database.csv file. 
city_data_df = pd.read_csv("../Weather_Database/WeatherPy_database.csv")
city_data_df.head()

,City_ID,City,Country,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Current Description
0,0,Mar Del Plata,AR,-38.00,-57.56,69.01,93,91,6.93,overcast clouds
1,1,Kedrovyy,RU,56.17,91.82,-25.60,67,78,4.47,mist
2,2,Rikitea,PF,-23.12,-134.97,78.76,78,100,14.09,moderate rain
3,3,Oranjemund,NaN,-28.55,16.43,66.07,68,100,13.56,overcast clouds
4,4,Safaga,EG,26.73,33.94,64.40,59,50,9.17,scattered clouds


In [3]:
# 2. Prompt the user to enter minimum and maximum temperature criteria 
min_temp = float(input("What is the minimum temperature you would like for your trip? "))
max_temp = float(input("What is the maximum temperature you would like for your trip? "))

What is the minimum temperature you would like for your trip? 75
What is the maximum temperature you would like for your trip? 85


In [4]:
# 3. Filter the city_data_df DataFrame using the input statements to create a new DataFrame using the loc method.
preferred_cities_df = city_data_df.loc[(city_data_df["Max Temp"] <= max_temp) & \
                                       (city_data_df["Max Temp"] >= min_temp)]
preferred_cities_df.head(10)


,City_ID,City,Country,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Current Description
2,2,Rikitea,PF,-23.12,-134.97,78.76,78,100,14.09,moderate rain
9,9,Souillac,MU,-20.52,57.52,80.60,74,75,14.99,light rain
16,16,Saint-Pierre,RE,-21.34,55.48,84.99,62,0,26.40,clear sky
18,18,Cap Malheureux,MU,-19.98,57.61,82.99,71,35,11.01,scattered clouds
27,27,Bathsheba,BB,13.22,-59.52,77.00,88,40,2.24,scattered clouds
29,29,Port Blair,IN,11.67,92.75,82.78,72,93,11.95,overcast clouds
30,30,Kruisfontein,ZA,-34.00,24.73,80.01,59,32,5.01,scattered clouds
33,33,Bambous Virieux,MU,-20.34,57.76,82.99,74,75,14.99,light rain
36,36,East London,ZA,-33.02,27.91,77.00,73,0,9.17,clear sky
38,38,Butaritari,KI,3.07,172.79,82.08,81,78,19.62,broken clouds


In [5]:
# 4a. Determine if there are any empty rows.
preferred_cities_df.count()


City_ID                150
City                   150
Country                147
Lat                    150
Lng                    150
Max Temp               150
Humidity               150
Cloudiness             150
Wind Speed             150
Current Description    150
dtype: int64

In [6]:
# 4b. Drop any empty rows and create a new DataFrame that doesn’t have empty rows.
clean_df=preferred_cities_df.dropna() 
clean_df

,City_ID,City,Country,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Current Description
2,2,Rikitea,PF,-23.12,-134.97,78.76,78,100,14.09,moderate rain
9,9,Souillac,MU,-20.52,57.52,80.60,74,75,14.99,light rain
16,16,Saint-Pierre,RE,-21.34,55.48,84.99,62,0,26.40,clear sky
18,18,Cap Malheureux,MU,-19.98,57.61,82.99,71,35,11.01,scattered clouds
27,27,Bathsheba,BB,13.22,-59.52,77.00,88,40,2.24,scattered clouds
...,...,...,...,...,...,...,...,...,...,...
669,669,Sabang,ID,5.89,95.32,82.02,79,45,14.05,light rain
671,671,Bubaque,GW,11.28,-15.83,77.67,58,0,16.96,clear sky
676,676,Kununurra,AU,-15.77,128.73,84.20,83,100,4.70,light rain
679,679,Magadi,IN,12.97,77.23,80.60,47,40,4.70,scattered clouds


In [7]:
# 5a. Create DataFrame called hotel_df to store hotel names along with city, country, max temp, and coordinates.
hotel_df = clean_df[["City", "Country", "Max Temp", "Current Description", "Lat", "Lng"]].copy()

# 5b. Create a new column "Hotel Name"
hotel_df["Hotel Name"] = ""
hotel_df.head(10)

,City,Country,Max Temp,Current Description,Lat,Lng,Hotel Name
2,Rikitea,PF,78.76,moderate rain,-23.12,-134.97,
9,Souillac,MU,80.60,light rain,-20.52,57.52,
16,Saint-Pierre,RE,84.99,clear sky,-21.34,55.48,
18,Cap Malheureux,MU,82.99,scattered clouds,-19.98,57.61,
27,Bathsheba,BB,77.00,scattered clouds,13.22,-59.52,
29,Port Blair,IN,82.78,overcast clouds,11.67,92.75,
30,Kruisfontein,ZA,80.01,scattered clouds,-34.00,24.73,
33,Bambous Virieux,MU,82.99,light rain,-20.34,57.76,
36,East London,ZA,77.00,clear sky,-33.02,27.91,
38,Butaritari,KI,82.08,broken clouds,3.07,172.79,


In [8]:
# 6a. Set parameters to search for hotels with 5000 meters.
params = {
    "radius": 5000,
    "type": "lodging",
    "key": g_key
}

# 6b. Iterate through the hotel DataFrame.
for index, row in hotel_df.iterrows():
    # 6c. Get latitude and longitude from DataFrame
    lat = row["Lat"]
    lng = row["Lng"]
     #add the latitude and longitude to location key for the params dictionary.
    params["location"] = f"{lat},{lng}"
    
    # 6d. Set up the base URL for the Google Directions API to get JSON data.
    base_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"

    # 6e. Make request and retrieve the JSON data from the search. 
    hotels = requests.get(base_url, params=params).json() 
    
    # 6f. Get the first hotel from the results and store the name, if a hotel isn't found skip the city.
    try:
        hotel_df.loc[index, "Hotel Name"] = hotels["results"][0]["name"]
    except (IndexError):
        print("Hotel not found... skipping.")
        

Hotel not found... skipping.
Hotel not found... skipping.


In [9]:
hotel_df


,City,Country,Max Temp,Current Description,Lat,Lng,Hotel Name
2,Rikitea,PF,78.76,moderate rain,-23.12,-134.97,Pension Maro'i
9,Souillac,MU,80.60,light rain,-20.52,57.52,Bils' Villa
16,Saint-Pierre,RE,84.99,clear sky,-21.34,55.48,Lindsey Hôtel
18,Cap Malheureux,MU,82.99,scattered clouds,-19.98,57.61,Pereybere Hotel & Spa
27,Bathsheba,BB,77.00,scattered clouds,13.22,-59.52,Atlantis Hotel
...,...,...,...,...,...,...,...
669,Sabang,ID,82.02,light rain,5.89,95.32,Hotel Citra
671,Bubaque,GW,77.67,clear sky,11.28,-15.83,"Casa Dora, Bubaque"
676,Kununurra,AU,84.20,light rain,-15.77,128.73,Hotel Kununurra
679,Magadi,IN,80.60,scattered clouds,12.97,77.23,Mindri of Magadi


In [10]:
# 7. Drop the rows where there is no Hotel Name.
clean_hotel_df = hotel_df.loc[hotel_df["Hotel Name"]!=""]
clean_hotel_df

,City,Country,Max Temp,Current Description,Lat,Lng,Hotel Name
2,Rikitea,PF,78.76,moderate rain,-23.12,-134.97,Pension Maro'i
9,Souillac,MU,80.60,light rain,-20.52,57.52,Bils' Villa
16,Saint-Pierre,RE,84.99,clear sky,-21.34,55.48,Lindsey Hôtel
18,Cap Malheureux,MU,82.99,scattered clouds,-19.98,57.61,Pereybere Hotel & Spa
27,Bathsheba,BB,77.00,scattered clouds,13.22,-59.52,Atlantis Hotel
...,...,...,...,...,...,...,...
669,Sabang,ID,82.02,light rain,5.89,95.32,Hotel Citra
671,Bubaque,GW,77.67,clear sky,11.28,-15.83,"Casa Dora, Bubaque"
676,Kununurra,AU,84.20,light rain,-15.77,128.73,Hotel Kununurra
679,Magadi,IN,80.60,scattered clouds,12.97,77.23,Mindri of Magadi


In [11]:
# 8a. Create the output File (CSV)

output_data_file ="../Vacation_Search/WeatherPy_vacation.csv"
# 8b. Export the City_Data into a csv
clean_hotel_df.to_csv(output_data_file, index_label="City_ID")

In [18]:
# 9. Using the template add city name, the country code, the weather description and maximum temperature for the city.
info_box_template = """
<dl>
<dt>Hotel Name</dt><dd>{Hotel Name}</dd>
<dt>City</dt><dd>{City}</dd>
<dt>Country</dt><dd>{Country}</dd>
<dt>Current Weather</dt><dd>{Current Description} and {Max Temp} °F</dd>

</dl>

"""

# 10a. Get the data from each row and add it to the formatting template and store the data in a list.
hotel_info = [info_box_template.format(**row) for index, row in clean_hotel_df.iterrows()]

# 10b. Get the latitude and longitude from each row and store in a new DataFrame.
locations = clean_hotel_df[["Lat", "Lng"]]

In [19]:
# 11a. Add a marker layer for each city to the map. 
locations = clean_hotel_df[["Lat", "Lng"]]

fig = gmaps.figure(center=(30.0, 31.0), zoom_level=1.5)

marker_layer = gmaps.marker_layer(locations, info_box_content=hotel_info)

fig.add_layer(marker_layer)
# 11b. Display the figure
fig

Figure(layout=FigureLayout(height='420px'))